In [1]:
import pandas as pd 
from functools import reduce
import pickle
import time
from collections import defaultdict
import os

# first_block = 389646
# last_block = 401221

with open('../pickles/ml/train_blocks', 'rb') as f:
    block_list = pickle.load(f)

#block_list = [400000]
    
#df = pd.read_pickle('../pickles/df/{}_to_{}.pickle'.format(first_block,last_block))
df = pd.read_pickle('../pickles/df/all_blocks.pickle')
#df = pd.read_pickle('../pickles/df/400000.pickle')
#df = pd.read_csv('./csv/400000addr.csv')
#df[[col for col in df.columns if not 'Unnamed' in col]]

# New columns for number of input and output transaction ids
df['num_txo'] = df.groupby('id_t')['id_txo_out'].transform('nunique')
df['num_txi'] = df.groupby('id_t')['id_txi'].transform('nunique')

/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [68]:
class User:
    def __init__(self, adrs, send_txs, rec_txs):
        self.adr = set(adrs)
        self.sending_tx = set(send_txs)
        #self.cadr = set()
        if rec_txs is None:
            self.receiving_tx = set()
        else:
            self.receiving_tx = set(rec_txs)
            
def query_database(query):
    # REMEMBER TO BE CONNECTED TO IMPERIAL WIFI!
    graph_db = py2neo.Graph("https://dsi-bitcoin.doc.ic.ac.uk:7473/db/data/", auth=("adi", "aditi123"))
    return graph_db.run(query)
        
#CHANGE LATER TO QUERY DATABASE INSTEAD
def iadrs_from_tx(id_t):
    return set(df['iadr'][df["id_t"] == id_t])
#     query_string = """
#     MATCH (t:Tx) <-[:IN]- (txi:TxIn) <-[:UNLOCK]- (a:Address) WHERE ID(t) = {}
#     RETURN a.address as iadr
#     """.format(id_t)
#     x = query_database(query_string).to_data_frame()
#     return set(x['iadr'])

def oadrs_from_tx(id_t):
    return set(df['oadr'][df["id_t"] == id_t])
#     query_string = """
#     MATCH (t:Tx) -[:OUT]-> (txo:TxOut) -[:LOCK]-> (a:Address) WHERE ID(t) = 113001822
#     RETURN a.address as oadr
#     """.format(id_t)
#     x = query_database(query_string).to_data_frame()
#     return set(x['oadr'])

def tx_from_iadr(iadr):
    return set(df['id_t'][df["iadr"] == iadr])

def tx_from_oadr(oadr):
    return set(df['id_t'][df["oadr"] == oadr])

def cadr_from_tx(tx_id, iadrs, potential_cadrs):
    count = 0
    o = set()
    for oadr in oadrs_from_tx(tx_id):
        if oadr in potential_cadrs and oadr not in iadrs:
            count = count + 1
            potential_cadrs.remove(oadr) #Ensure that same change address won't be another user
            o.add(oadr)
    if count>1:
        print("NOOOO")
    return o, potential_cadrs

#FUNCTION TO WHICH YOU GIVE AN INPUT ADDRESS AND GET USER 
def get_user(input_adr, potential_cadrs):
    to_inv = [input_adr]
    user_iadrs = set()
    seen_txs = set()
    while to_inv:
        current_iadr = to_inv.pop(0)
        user_iadrs.add(current_iadr)
        for id_t in tx_from_iadr(current_iadr):
            
            if id_t not in seen_txs:
                seen_txs.add(id_t)
                iadrs = iadrs_from_tx(id_t)
                cadr, potential_cadrs = cadr_from_tx(id_t, iadrs, potential_cadrs)
                iadrs.update(cadr)
                to_inv += iadrs.difference(user_iadrs) #Adding addr)
                user_iadrs.update(iadrs)  
    print(seen_txs)
    return User(user_iadrs, seen_txs, None), potential_cadrs

#TAKES OUTPUT ADDRESS AND GIVES USER THAT HAS THAT ADDRESS AS INPUT
def user_from_oadr(oadr):
    for i,user in enumerate(users):
        if oadr in user.adr:
            return i        

# def get_cadr(user, potential_cadrs):
#     cadrs_for_user = set()
#     for tx_id in user.sending_tx:    #CHECK THIS
#         o = oadrs_from_tx(tx_id)
#         for oadr in o:
#             if oadr in potential_cadrs and oadr not in user.adr:
#                 user.adr.add(oadr)
#                 potential_cadrs.remove(oadr) #Ensure that same change address won't be another user
#     return user, potential_cadrs

## Heuristic 1 & 2

In [3]:
otc_dic = {}
for block in block_list:
#for block in range(first_block,last_block+1,1):
    if(os.path.exists("../pickles/df/{}.pickle".format(block)) and os.path.exists("../pickles/otc/otc_{}.pickle".format(block))):
        with open ('../pickles/otc/otc_{}.pickle'.format(block), 'rb') as fp:
            otc_dic[block] = pickle.load(fp)
    
not_seen = list(reduce(set.symmetric_difference, (set(val) for val in otc_dic.values())))

In [69]:
potential_cadrs = []
tx_to_check = []
blah = []
for a in not_seen:
    b = tx_from_oadr(a)
    c = tx_from_iadr(a)
    if not c and (len(b)==1):
        tx_to_check.extend(list(b))
        blah.extend(list(c))

KeyboardInterrupt: 

In [88]:
len(not_seen)

71428

In [67]:
appeared_once_o= list(df.drop_duplicates(['id_txo_out']).oadr.value_counts()[df.drop_duplicates(['id_txo_out']).oadr.value_counts()==1].index)
all_iadrs= list(df.iadr.value_counts().index)
o_never_used_as_i = set(appeared_once_o).difference(all_iadrs)

#Change Transactions
for i,user in enumerate(users):
    cadrs_for_user = set()
    for tx_id in user.sending_tx:    #CHECK THIS
        o = oadrs_from_tx(tx_id)
        potential_cadr = []
        for oadr in o:
            if oadr in not_seen and oadr not in user.adr and oadr in o_never_used_as_i:
                potential_cadr.append(oadr)
        if len(potential_cadr)==1:
            cadrs_for_user.add(potential_cadr[0])
            not_seen.remove(potential_cadr[0]) #Ensure that same change address won't be another user
    user.adr.update(cadrs_for_user)   

56796

In [65]:
#LIST OF LISTS OF USER AND THEIR ASSOCIATED ADDRESSES
users = []
pc = list(potential_cadrs)  #Reset potential cadrs

starttime = time.time()        
        
# Bitcoin- 
bitcoin = User({str(0)}, set(), set()) #Make user object with bitcoin iadr (which is 0)
seen_miner_iadrs_tx = defaultdict(set) #Make dict associating miner payment address with tx_ids they've been involved in

for index, row in df[df['iadr'] == str(0)].iterrows(): #Going through all mining txs
    bitcoin.sending_tx.add(row['id_t']) #Adding tx id to bitcoin user's txs
    seen_miner_iadrs_tx[row['oadr']].add(row['id_t']) # Updating dict to register the tx_id as corresponding to miners adr. If new, then new key added, otherwise added to values of existing key

users.append(bitcoin) # Add bitcoin user
already_seen_adr = {'0'}  # Bitcoin iadr has already been seen
print("make miner users")
# make miners users
for adr, id_ts in seen_miner_iadrs_tx.items(): #Go through dictionary for every adr (miner) and txs he's been involved
    print(adr)
    miner, pc = get_user(adr, pc) #From an address, give back user ... aka identify all addresses belonging to miner
    miner.adr.add(adr) #Make sure adrs and txs are added in
    miner.receiving_tx.update(id_ts)
    users.append(miner)
    already_seen_adr.update(miner.adr)#Made sure miner's addresses are in already seen so that we don't create a second user with the same addresses
    break
print("make other users")
# ASSOCIATE INPUT ADDRESS AND TX WITH EACH USER IN BLOCK
#make other users from heuristic
# count = 0
# for input_adr in df.iadr:
#     if count == 12:
#         break
#     if input_adr not in already_seen_adr:
#         user, pc = get_user(input_adr, pc)
#         #user, potential_cadrs = get_cadr(user, potential_cadrs)
#         users.append(user)
#         already_seen_adr.update(user.adr)
#         count +=1
        
# edges = defaultdict(int)

# print("Which users have transacted with each other")
# #WHICH USERS IN HAVE TRANSACTED WITH EACH OTHER
# for i,user in enumerate(users):
#     for tx_id in user.sending_tx:
#         for oadr in oadrs_from_tx(tx_id):
#             if oadr in already_seen_adr:
#                     edges[(i, user_from_oadr(oadr))]+=1  
                    
# print("Total time to process heuristics:", time.time()-starttime)

make miner users
1KM7w12SkjzJ1FYV2g1UCMzHjv3pkMgkEb
{113260442, 113512733}
make other users


In [7]:
with open('../pickles/pool/all_users.pickle','wb') as f:    
    pickle.dump(users,f)

In [ ]:
with open('./users_2.pickle', 'rb') as f:
    users2 = pickle.load(f)
for i, user in enumerate(users):
    for j, u in enumerate(users2):
        if user.adr!=u.adr and i==j:
            print('1')
            print(i,j)
        if user.sending_tx!=u.sending_tx and i==j:
            print('2')
            print(i,j)
        if user.receiving_tx!=u.receiving_tx and i==j:
            print('3')
            print(i,j)

In [ ]:
def getCouples(users):
    to_combine = []
    for i, user in enumerate(users):
        for k, otheruser in enumerate(users):
            if user.adr.intersection(otheruser.adr) and i!=k:
                to_combine.append((i,k))
    return to_combine

print(len(users))
to_combine = getCouples(users) 
while to_combine:
    print("to_combine length ", len(to_combine))
    st = set(users)
    for tple in to_combine:
        user1 = users[tple[0]]
        user2 = users[tple[1]]
        if user1 in st and user2 in st:
            #print(tple)
            st.remove(user1)
            st.remove(user2)
            user1.adr = user1.adr.union(user2.adr)
            st.add(user1)
    users = list(st)
    print(len(users))
    to_combine = getCouples(users)

In [ ]:
#Save Users found
#with open('../pickles/users/users_{}_to_{}.pickle'.format(first_block,last_block),'wb') as f:
with open('../pickles/pool/all_users.pickle','wb') as f:    
    pickle.dump(users,f)

In [17]:
count = 0
for i, user in enumerate(users):
    for k, otheruser in enumerate(users):
        if (user.sending_tx.intersection(otheruser.sending_tx) or user.receiving_tx.intersection(otheruser.receiving_tx)) and i!=k:
            print(i,k)

1 6
1 9
1 11
6 1
6 9
6 11
7 13
9 1
9 6
9 11
11 1
11 6
11 9
13 7


KeyboardInterrupt: 

In [39]:
users[11].sending_tx

{113260442, 113512733}

In [25]:
users[6].sending_tx

{113260442, 113512733}

In [38]:
users[11].receiving_tx

{113260436, 113767288}

In [27]:
users[6].receiving_tx

{113072104}

In [23]:
users[1].adr

{'15HCzh8AoKRnTWMtmgAsT9TKUPrQ6oh9HQ',
 '15urYnyeJe3gwbGJ74wcX89Tz7ZtsFDVew',
 '16ezpwa2dbuTbH79Tjg9ZzLuogncWDw6kp',
 '1B2oHUNjQKzwZ3BbybwqPUhfYg8CjmvPcL',
 '1Ebb8NfVmKMoGuMJCAEbVMv2dX8GnzgxSa',
 '1F1MAvhTKg2VG29w8cXsiSN2PJ8gSsrJw',
 '1JVMpaWzLB1pnkwoQNqK4Z4EimQj5bCKj7',
 '1JdgLJewU2fStn2xYYcts4KNPRibtv1ZLJ',
 '1KM7w12SkjzJ1FYV2g1UCMzHjv3pkMgkEb',
 '1M5hoG1pCTDsPqZwG6WH25ziwYYaXNMLrU',
 '1PE7LXcntsLvavM2KXpJGNu51UbDhC3u63'}

In [22]:
users[6].adr

{'15HCzh8AoKRnTWMtmgAsT9TKUPrQ6oh9HQ',
 '15urYnyeJe3gwbGJ74wcX89Tz7ZtsFDVew',
 '1B2oHUNjQKzwZ3BbybwqPUhfYg8CjmvPcL',
 '1Ebb8NfVmKMoGuMJCAEbVMv2dX8GnzgxSa',
 '1F1MAvhTKg2VG29w8cXsiSN2PJ8gSsrJw',
 '1JVMpaWzLB1pnkwoQNqK4Z4EimQj5bCKj7',
 '1KM7w12SkjzJ1FYV2g1UCMzHjv3pkMgkEb',
 '1M5hoG1pCTDsPqZwG6WH25ziwYYaXNMLrU',
 '1PE7LXcntsLvavM2KXpJGNu51UbDhC3u63'}

In [28]:
users[11].adr

{'15HCzh8AoKRnTWMtmgAsT9TKUPrQ6oh9HQ',
 '15urYnyeJe3gwbGJ74wcX89Tz7ZtsFDVew',
 '1B2oHUNjQKzwZ3BbybwqPUhfYg8CjmvPcL',
 '1Ebb8NfVmKMoGuMJCAEbVMv2dX8GnzgxSa',
 '1F1MAvhTKg2VG29w8cXsiSN2PJ8gSsrJw',
 '1JVMpaWzLB1pnkwoQNqK4Z4EimQj5bCKj7',
 '1KM7w12SkjzJ1FYV2g1UCMzHjv3pkMgkEb',
 '1M5hoG1pCTDsPqZwG6WH25ziwYYaXNMLrU',
 '1PE7LXcntsLvavM2KXpJGNu51UbDhC3u63'}

In [29]:
users[11].sending_tx

{113260442, 113512733}

In [30]:
users[11].receiving_tx

{113260436, 113767288}

In [11]:
from collections import Counter

tx_ids = []
for user in users[1:]:#Drop out bitcoin user
    tx_ids += list(user.sending_tx)

#tx_ids = list.append([list(user.sending_tx) for user in users])
        
ads = []
for i,user in enumerate(users):
    for ad in user.adr:
        ads.append(ad)
        
# cads = []
# for i,user in enumerate(users):
#     for cad in user.cadr:
#         cads.append(cad)

if Counter(tx_ids).most_common(10)[0][1] == 1:  ##Repeated txids between bitcoin and the miners
    print("No repeats txid")
    
if Counter(ads).most_common(10)[0][1] == 1:
    print("No repeats ads")

In [ ]:
# #Read Users found
# with open('../pickles/users/users_{}_to_{}.pickle'.format(first_block,last_block), 'rb') as f:
#     users = pickle.load(f)

In [ ]:
#df.groupby('id_t').agg({'oadr':['nunique', 'count'],'iadr':['nunique', 'count'],'id_txo_in':['nunique', 'count'],'id_txo_out':['nunique', 'count']})
#df.to_csv('users.csv', columns=['input_user','output_user'])

## Interactions between users (constructing user graph)

In [ ]:
#Construct User Graph
df['input_user'] = df['iadr']  
df['output_user'] = df['oadr']

starttime = time.time() 
#Replacing all input addresses and output addresses with a user corresponding to that address
for i, user in enumerate(users): 
    #assert(isinstance(i, int))
    df['input_user'] = df['input_user'].apply(lambda x: i if x in user.adr else x)
    df['output_user'] = df['output_user'].apply(lambda x: i if x in user.adr else x)
    
for tx_id, output_user in df[['id_t','output_user']].values:
    if isinstance(output_user,int):
        users[output_user].receiving_tx.add(tx_id)
print("Total time to construct user graph:", time.time()-starttime)

#df.to_pickle("../pickles/df/{}_to_{}_users.pickle".format(first_block,last_block))
df.to_pickle("../pickles/pool/all_df_with_users.pickle")
#Save Users found
#with open('../pickles/users/users_{}_to_{}.pickle'.format(first_block,last_block),'wb') as f:
with open('../pickles/pool/all_users.pickle','wb') as f: 
    pickle.dump(users,f)
    
# can't trust input_val column now
# because dropped lots of inputs
edges_df0 = df.drop_duplicates(['input_user', 'id_txo_out'])
edges_df = edges_df0.groupby(['input_user', 'output_user']).apply(lambda group: group['output_val'].sum()).reset_index()
#edges_df0 = edges_df0.rename(columns={0: 'edge_amount'})

tups = [(input_user, output_user, amount) for (index, input_user, output_user, amount) in edges_df.itertuples()]

#with open("../pickles/user_graphs/{}_to_{}_users.pickle".format(first_block,last_block), 'wb') as f:
with open('../pickles/pool/all_users_graph.pickle','wb') as f: 
    pickle.dump(tups,f)
#tups2 = [(input_user, output_user, amount) for (index, input_user, output_user, amount) in edges_df2.itertuples()]

# assert(all(tups == tups2))


In [ ]:
len(users)

In [ ]:
# df.head()
# df['iadr'].apply(type).value_counts()